# Temporal-Aware RAG Example

This notebook demonstrates the Temporal-Aware RAG architecture that considers time-based information in both retrieval and generation processes. It handles time-sensitive queries and retrieves documents based on temporal relevance, recency, and historical context.

In [ ]:
# Import required modules
import numpy as np
import hashlib
from datetime import datetime, timedelta
from src.rag_specialized.temporal_aware.temporal_aware_rag import (
    TemporalAwareRAG, TemporalDocument, TemporalQuery, TemporalScope
)

## Initialize the Temporal-Aware RAG System

In [ ]:
# Initialize the Temporal-Aware RAG system
temporal_rag = TemporalAwareRAG(temporal_weight=0.4)
print("Temporal-Aware RAG system initialized!")

## Create Sample Temporal Documents

In [ ]:
# Create sample temporal documents with different timestamps
now = datetime.now()

documents = [
    TemporalDocument(
        id="doc1",
        content="The company reported record profits in Q4 2023, with revenues reaching $1.2 billion.",
        timestamp=now - timedelta(days=30),  # 1 month ago
        metadata={"source": "financial_report", "quarter": "Q4_2023", "revenue": "$1.2B"}
    ),
    TemporalDocument(
        id="doc2",
        content="Market analysis from early 2023 showed steady growth across all sectors.",
        timestamp=now - timedelta(days=300),  # ~10 months ago
        metadata={"source": "market_analysis", "period": "early_2023", "growth": "steady"}
    ),
    TemporalDocument(
        id="doc3",
        content="The new product launch scheduled for next quarter is expected to drive significant growth.",
        timestamp=now + timedelta(days=90),  # 3 months in future
        metadata={"source": "planning_doc", "event": "product_launch", "expected_impact": "significant_growth"}
    ),
    TemporalDocument(
        id="doc4",
        content="Historical data from 2020 shows the impact of global events on market stability.",
        timestamp=now - timedelta(days=2200),  # ~6 years ago
        metadata={"source": "historical_data", "year": "2020", "impact": "market_volatility"}
    )
]

print(f"Created {len(documents)} sample temporal documents")
for i, doc in enumerate(documents):
    print(f"  Doc {i+1}: {doc.timestamp.strftime('%Y-%m-%d')} - {doc.content[:50]}...")

## Add Documents to the RAG System

In [ ]:
# Add documents to the system
num_added = temporal_rag.add_documents(documents)
print(f"Added {num_added} temporal documents to the system")

## Create and Execute Temporal Queries

In [ ]:
# Create a recent-focused query
recent_query = TemporalQuery(
    text="What were the recent financial results?",
    reference_time=now,
    temporal_scope=TemporalScope.RECENT,
    recency_bias=0.8,
    time_window_days=60  # Only consider last 60 days
)

# Create a simple embedding for the query
query_text_hash = hashlib.md5(recent_query.text.encode()).hexdigest()
query_embedding = np.frombuffer(bytes.fromhex(query_text_hash[:32]), dtype=np.float32)
if len(query_embedding) < 384:
    query_embedding = np.pad(query_embedding, (0, 384 - len(query_embedding)), 'constant')
elif len(query_embedding) > 384:
    query_embedding = query_embedding[:384]

# Execute the query
recent_result = temporal_rag.query(recent_query, query_embedding, k=2)

print("Recent Financial Results Query:")
print(f"Query: {recent_query.text}")
print(f"Answer: {recent_result.answer}")
print(f"Temporal context: {recent_result.temporal_context}")
print(f"Temporal accuracy: {recent_result.temporal_accuracy:.3f}")
print(f"Confidence: {recent_result.confidence:.3f}")
print(f"Latency: {recent_result.latency_ms:.2f}ms")
print(f"Sources: {len(recent_result.sources)} documents retrieved")

In [ ]:
# Create a historical-focused query
historical_query = TemporalQuery(
    text="What happened in 2020?",
    reference_time=now,
    temporal_scope=TemporalScope.HISTORICAL,
    recency_bias=0.1  # Low recency bias for historical focus
)

# Create a simple embedding for the query
hist_query_hash = hashlib.md5(historical_query.text.encode()).hexdigest()
hist_query_embedding = np.frombuffer(bytes.fromhex(hist_query_hash[:32]), dtype=np.float32)
if len(hist_query_embedding) < 384:
    hist_query_embedding = np.pad(hist_query_embedding, (0, 384 - len(hist_query_embedding)), 'constant')
elif len(hist_query_embedding) > 384:
    hist_query_embedding = hist_query_embedding[:384]

# Execute the query
hist_result = temporal_rag.query(historical_query, hist_query_embedding, k=2)

print("\nHistorical Query:")
print(f"Query: {historical_query.text}")
print(f"Answer: {hist_result.answer}")
print(f"Temporal context: {hist_result.temporal_context}")
print(f"Sources: {len(hist_result.sources)} documents retrieved")

In [ ]:
# Create a future-focused query
future_query = TemporalQuery(
    text="What are upcoming events?",
    reference_time=now,
    temporal_scope=TemporalScope.FUTURE,
    recency_bias=0.2
)

# Create a simple embedding for the query
future_query_hash = hashlib.md5(future_query.text.encode()).hexdigest()
future_query_embedding = np.frombuffer(bytes.fromhex(future_query_hash[:32]), dtype=np.float32)
if len(future_query_embedding) < 384:
    future_query_embedding = np.pad(future_query_embedding, (0, 384 - len(future_query_embedding)), 'constant')
elif len(future_query_embedding) > 384:
    future_query_embedding = future_query_embedding[:384]

# Execute the query
future_result = temporal_rag.query(future_query, future_query_embedding, k=2)

print("\nFuture Events Query:")
print(f"Query: {future_query.text}")
print(f"Answer: {future_result.answer}")
print(f"Temporal context: {future_result.temporal_context}")
print(f"Sources: {len(future_result.sources)} documents retrieved")

## Explore the System's Internal State

In [ ]:
# Examine the retriever's internal state
print(f"Number of documents in system: {len(temporal_rag.retriever.documents)}")
print(f"Embedding matrix shape: {temporal_rag.retriever.embeddings.shape if temporal_rag.retriever.embeddings is not None else 'None'}")
print(f"Temporal weight: {temporal_rag.retriever.temporal_weight}")

# Display document information
for i, doc in enumerate(temporal_rag.retriever.documents):
    age_days = (now - doc.timestamp).days
    print(f"Document {i+1}: {doc.id} - {doc.timestamp.strftime('%Y-%m-%d')} ({age_days} days old) - {len(doc.content)} chars")

## Performance Analysis

In [ ]:
# Perform multiple queries to analyze performance
queries = [
    ("recent", "What were the recent financial results?"),
    ("historical", "What happened in early 2023?"),
    ("general", "Tell me about company performance"),
    ("future", "What are upcoming initiatives?")
]

latencies = []
confidences = []
temporal_accuracies = []

for scope, query_text in queries:
    # Create temporal query based on scope
    if scope == "recent":
        query = TemporalQuery(
            text=query_text,
            reference_time=now,
            temporal_scope=TemporalScope.RECENT,
            recency_bias=0.8
        )
    elif scope == "historical":
        query = TemporalQuery(
            text=query_text,
            reference_time=now,
            temporal_scope=TemporalScope.HISTORICAL,
            recency_bias=0.1
        )
    elif scope == "future":
        query = TemporalQuery(
            text=query_text,
            reference_time=now,
            temporal_scope=TemporalScope.FUTURE,
            recency_bias=0.2
        )
    else:
        query = TemporalQuery(
            text=query_text,
            reference_time=now,
            temporal_scope=TemporalScope.ALL_TIME,
            recency_bias=0.5
        )
    
    # Create embedding
    query_hash = hashlib.md5(query_text.encode()).hexdigest()
    query_emb = np.frombuffer(bytes.fromhex(query_hash[:32]), dtype=np.float32)
    if len(query_emb) < 384:
        query_emb = np.pad(query_emb, (0, 384 - len(query_emb)), 'constant')
    elif len(query_emb) > 384:
        query_emb = query_emb[:384]
    
    result = temporal_rag.query(query, query_emb, k=2)
    latencies.append(result.latency_ms)
    confidences.append(result.confidence)
    temporal_accuracies.append(result.temporal_accuracy)

print(f"Average query latency: {np.mean(latencies):.2f}ms")
print(f"Latency std deviation: {np.std(latencies):.2f}ms")
print(f"Average confidence: {np.mean(confidences):.3f}")
print(f"Average temporal accuracy: {np.mean(temporal_accuracies):.3f}")

## Summary

In this notebook, we explored the Temporal-Aware RAG architecture:

1. **Initialization**: Created an instance of the TemporalAwareRAG system
2. **Temporal Documents**: Added documents with different timestamps (past, present, future)
3. **Temporal Queries**: Executed queries with different temporal scopes (recent, historical, future)
4. **System Analysis**: Examined the internal state and temporal characteristics
5. **Performance Evaluation**: Measured query latency, confidence, and temporal accuracy

The Temporal-Aware RAG system successfully processed time-sensitive queries and returned relevant responses while considering the temporal context of both documents and queries.